<a href="https://colab.research.google.com/github/pietroventurini/machine-learning-notes/blob/master/1%20-%20Concept%20Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Concept Learning
### Contents

1. **Introduction**  
    1.1 Prototypical concept learning task  
    1.2 The inductive learning hypothesis  
    1.3 Concept learning as search  
    1.4 General-to-specific ordering of hypotheses  
2. **Concept learning as search**  
    2.1. Find-S algorithm  
    2.3. Candidate elimination algorithm  
    2.4. List-then-eliminate algorithm    
5. **Biased and unbiased learners** 
6. **Overfitting**

## Introduction
**Concept learning** consists into inferring a boolean-valued function from training examples of its input and output. In other words, it can be formulated as the problem of searching through a predefined space of potential hypotheses for the hypothesis that best fits the training examples. A **hypothesis** is described by a conjuction of constraints on the attributes of the problem's instances.

Consider the following set of examples, where `EnjoySport` indicates whether or not someone enjoys his favorite water sport on that day. The task is to learn to predict the value of `EnjoySport` for an arbitrary day, based on the values of its other attributes.

| Example | Sky   | AirTemp | Humidity | Wind   | Water | Forecast | EnjoySport |
|---------|-------|---------|----------|--------|-------|----------|------------|
| 1       | Sunny | Warm    | Normal   | Strong | Warm  | Same     | Yes        |
| 2       | Sunny | Warm    | High     | Strong | Warm  | Same     | Yes        |
| 3       | Rainy | Cold    | High     | Strong | Warm  | Change   | No         |
| 4       | Sunny | Warm    | High     | Strong | Cool  | Change   | Yes        |

Let each hypothesis be a vector of the six attributes *[Sky, AirTemp, Humidity, Wind, Water, Forecast]*. For each of them the hypothesis will either:
- indicate by `?` that any value is acceptable for that attribute
- specify a single required value
- indicate by `Ø` that no value is acceptable

An instance $x$ for which all the constraints of a certain hypothesis are satisfied, will be classified as positive, meaning $h(x)=1$. For instance, according to  hypothesis $\langle ?, Cold,High,?,?,?\rangle$, only the third example would be classified as positive. The most general hypothesis is $\langle?,?,?,?,?,?\rangle$, while the most specific one is $\langle\emptyset,\emptyset,\emptyset,\emptyset,\emptyset,\emptyset\rangle$.

### Prototypical concept learning task

**Given**:
- **Instances** $X$: the possible days, represented by the six attributes,
- **Hypotheses** $H$: each described by a conjunction of constraints on the attributes,
- **Target concept** $c$: the function $\text{EnjoySport}: X\rightarrow\{0,1\}$,
- **Training examples**: positive/negative examples of the target function, each of the form $\langle x,c(x) \rangle$.

**Determine**:
- A hypothesis $h \in H$ such that $h(x)=c(x) \; \forall x\in X$.

### The inductive learning hypothesis
In ***inductive learning*** our assumption is that the best hypothesis regarding unseen instances is the hypothesis that best fits the observed training data.

# Concept learning as search
The space of hypothesis, thus the space of all hypotheses that the program can ever learn, is implicitly defined by the hypothesis representation we choose. The goal of this search is to find the hypothesis that best fits the training examples.

In our example, the instance space $X$ contains exactly
$3\cdot2\cdot2\cdot2\cdot2\cdot2 = 96$ distinct instances. The number of *semantically distinct* hypotheses is  $1 + (4\cdot3\cdot3\cdot3\cdot3\cdot3) = 973$ (by considering the placeholder `?` and that every hypothesis containing one or more `Ø` symbols represents the empty set of instances, therefore it classifies every instance as negative. Practical learning tasks can involve infinite hypothesis spaces.

### General-to-specific ordering of hypotheses
Many algorithms for concept learning rely on the general-to-specific ordering of hypotheses. Consider this two hypotheses:
$$h_1=\langle Sunny, ?, ?, Strong, ?, ?\rangle$$
$$h_2=\langle Sunny, ?, ?, ?, ?, ?\rangle$$

Any instance classified positive by $h_1$ will also be classified positive by $h_2$. Therefore, we say that $h_2$ is more general than $h_1$.

**Definition:** Let $h_j$ and $h_k$ be boolean-valued functions defined over $X$. Then $h_j$ is ***more general-than-or-equal-to*** $h_k$ (written $h_j \ge_g h_k$) if and only if

$$(\forall x \in X)\left[ (h_k(x)=1)\rightarrow (h_j(x)=1)\right]$$


**Definition:** $h_j$ is ***strictly more general-than-or-equal-to*** $h_k$ (written $h_j >_g h_k$) if and only if

$$(h_j\ge_g h_k) \land (h_k \not \ge_g h_j)$$

#### Example
Consider the following three hypotheses from the EnjoySports example:

$$\begin{equation}
    \begin{split}
        &h_1 = \langle Sunny, ?, ?, Strong, ?, ?\rangle \\
        &h_2 = \langle Sunny, ?, ?, ?, ?, ?\rangle \\
        &h_3 = \langle Sunny, ?, ?, ?, Cool, ?\rangle
    \end{split}
\end{equation}$$

and the instances:

$$\begin{equation}
    \begin{split}
        &x_1 = \langle Sunny, Warm, High, Strong, Cool, Same\rangle \\
        &x_2 = \langle Sunny, Warm, High, Light, Warm, Same\rangle \\
    \end{split}
\end{equation}$$

It holds that $h_2 \ge_g h_1$ and $h_2\ge_g h_3$. Neither $h_1$ nor $h_3$ is more general than the other.

## FIND-S Algorithm
Find-S begins with the most specific possible hypothesis in $H$, then generalize this hypothesis each time it fails to cover an observed positive training example.

1. Initialize $h$ to the most specific hypothesis in $H$: $h\leftarrow \langle \emptyset,...,\emptyset\rangle$
2. For each positive training instance $x$:  
    - For each attribute constraint $a_i$ in $h$:  
        - *If* the constraint $a_i$ is satisfied by $x$ *then* do nothing *else* replace $a_i$ in $h$ by the next more general constraint that is satisfied by $x$.
3. Output hypothesis $h$

FIND-S is guaranteed to output the most specific hypothesis within $H$
that is consistent with the positive training examples. At each iteration, the current hypothesis is the most specific one which his consistent with the instances observed up to that point. 

Anyway, there are several questions left unanswered:

1. Has the learner learned the correct target concept? Although it will find a hypothesis consistent with the training data, it can't determine if it is the *only* hypothesis in $H$ consistent with the training data.  
2. Why we should prefer the most specific hypothesis?  
3. Are the training examples consistent? We would prefer an algorithm that could at least detect when the training data is inconsistent and, preferably, accommodate such errors.  
4. If there are many maximally specific consistent hypotheses, which one should we choose?

Given $D$, the FIND-S algorithm outputs the hypothesis $h=\langle Sunny, Warm, ?, Strong, ?, ?\rangle$ 

## CANDIDATE-ELIMINATION Algorithm
Differently from Find-S, which outputs just one of possibly many hypotheses consistent with the training set, CANDIDATE-ELIMINATION outputs a description of the set of all hypotheses consistent with the training examples.

**Definition:** A hypothesis $h$ is ***consistent*** with a set of training examples $D$ if and only if $h(x)=c(x)$ for each training example $\langle x,c(x)\rangle$ in $D$. 

$$ Consistent(h,D)\equiv (\forall \langle x, c(x)\rangle \in D) \; h(x)=c(x)$$

The CANDIDATE-ELIMINATION algorithm represents the set of all hypotheses consistent with the observed training examples. This subset is called the **version space** with respect to the hypothesis space $H$ and the training examples $D$.

**Definition:** The ***version space***, denoted $VS_{H,D}$, with respect to hypothesis space $H$ and training examples $D$, is the subset of hypotheses from $H$ consistent with the training examples in $D$.

$$VS_{H,D} \equiv \{h\in H \vert Consistent(h,D)\}$$

The CANDIDATE-ELIMINATION algorithm represents the version space by storing only its most general members (labeled `G`) and its most specific (labeled `S`). Given these two sets $S$ and $G$, we can enumerate all members of the version space as needed by generating the hypotheses that lie between these two sets in the general-to-specific partial ordering.

**Definition:** the ***general boundary*** $G$, with respect to hypothesis space $H$ and training data $D$, is the set of maximally general members of $H$ consistent with $D$.

$$G \equiv \{g \in H\; \vert\; Consistent(g,D) \land (\neg \exists g'\in H)[(g'>_g g) \land Consistent(g',D)]\}$$

**Definition:** the ***specific boundary*** $S$, with respect to hypothesis space $H$ and training data $D$, is the set of minimally general (i.e., maximally specific) members of $H$ consistent with $D$.

$$S \equiv \{s \in H\; \vert\; Consistent(s,D) \land (\neg \exists s'\in H)[(s>_g s') \land Consistent(s',D)]\}$$

**Version space representation theorem:** let $X$ be an arbitrary set of instances and let $H$ be a set of boolean-valued hypotheses defined over $X$. Let $c: X \rightarrow \{0, 1\}$ be an arbitrary target concept defined over $X$, and let $D$ be an arbitrary set of training examples $\{\langle x, c(x)\rangle\}$. For all $X$, $H$, $c$, and $D$ such that $S$ and $G$ are well defined,

$$VS_{H,D}=\{h\in H \vert (\exists s\in S)(\exists g \in G)(g\ge_g h \ge_g s)\}$$



### LIST-THEN-ELIMINATE Algorithm
One obvious way to represent the version space is simpy to list all of its members. The *LIST-THEN-ELIMINATE* algorithm first initializes the version space containing all the hypotheses in $H$ and then eliminates any hypothesis found inconsistent with any training example. It is guaranteed to output all hypotheses consistent with the training data, but it requires to enumerate all hypotheses in $H$, that is an unrealistic requirement.

1. $VersionSpace \leftarrow$ a list containing every hypothesis in $H$.
2. For each training example $\langle x,c(x)\rangle$ remove from $VersionSpace$ any hypothesis $h$ for which $h(x)\neq c(x)$
3. Output the list of hypotheses in $VersionSpace$

<img src="images/concept-learning/version-space.png" width="550"/>



## CANDIDATE-ELIMINATION Learning Algorithm
The candidate-elimination algorithm computes the version space containing all hypotheses from $H$ that are consistent with an observed sequence of training examples.
```python
Initialize G to the set of maximally general hypotheses in H (G := {?,?,?,?,?,?})
Initialize S to the set of maximally specific hypotheses in H (S := {Ø,Ø,Ø,Ø,Ø,Ø})

for each training example d, do:
  if d is a positive example:
    Remove from G any hypothesis inconsistent with d
    For each hypothesis s in S that is not consistent with d:
      - Remove s from S
      - Add to S all minimal generalizations h of s such that h is consistent with d, and some member of G is more general than h
      - Remove from S any hypothesis that is more general than another hypothesis in S
  if d is a negative example:
    - Remove from S any hypothesis inconsistent with d
    for each hypothesis g in G that is not consistent with d:
      - Remove g from G
      - Add to G all minimal specializations h of g such that h is consistent with d, and some member of S is more specific than h
      - Remove from G any hypothesis that is more specific than another hypothesis in G
```

As each training example is considered, the $S$ and $G$ boundary sets are generalized and specialized, respectively, to eliminate from the version space any hypotheses found inconsistent with the new training example. Positive training examples may force the $S$ boundary of the version space to become increasingly general. Negative training examples play the complimentary role of forcing the $G$ boundary to become increasingly specific. Any hypothesis more general than $S$ will, by definition, cover any example that $S$ covers and thus will cover any past positive example. Any hypothesis more specific than $G$ is assured to be consistent with past negative examples.

An illustrative example can be found at page 33 of *Machine Learning - Thomas Mitchell (McGraw Hill, 1997)*.

### Convergence of the CANDIDATE-ELIMINATION algorithm
The Candidate-Elimination Algorithm will converge toward the hypothesis that correctly describes the target concept if (necessary but not sufficient condition):
1. there are no errors in the training examples;
2. there is a hypothesis in $H$ that correctly describes the target concept.

The target concept is exactly learned when the $S$ and $G$ boundary sets converge to a single identical hypothesis.

The **optimal query strategy** for a concept learner (to ask to an oracle) is to generate instances that satisfy exactly **half** of the hypothesis in the currentversion space.

If this is possible, we can find the target concept with $\left\lceil {\log_2|VS|} \right \rceil$ queries.



## Classification of new instances
To classify a new instance as positive, it is sufficient to verify that the instance is classified as positive by **every** member of $S$.

To classify a new instance as negative, it is sufficient to verify that the instance is classified as negative by **every** member of $G$.

If there are some hypotheses that classify the instance as positive and some others that classify it as negative, we can take, for example, a ***majority vote***, selecting the most frequent outcome.

## Biased and Unbiased learner
Suppose we wish to assure that the hypothesis space contains the unknown target
concept. The obvious solution is to enrich the hypothesis space to include
every possible hypothesis. If we restrict the hypothesis space to include only conjunctions of attribute values, the hypothesis space is unable to represent
even simple disjunctive target concepts such as "*Sky = Sunny or Sky = Cloudy*". The choice of such a representation for the hypotheses represents the **inductive bias** of the learner.

The solution to the problem of assuring that the target concept is in the hypothesis space $H$ is to provide a hypothesis space capable of representing every teachable concept (the set of all subsets of $X$, i.e. the *power set* of $X$, with dimension $2^{|X|}$).

However, while this hypothesis space eliminates any problems of expressibility, it unfortunately raises a new, equally difficult problem: our concept learning algorithm is now completely unable to generalize beyond the observed examples (the $S$ boundary will always be simply the disjunction of the observed positive examples, while the $G$ boundary will always be the negated disjunction of the observed negative examples). 

**Property of inductive inference:** A learner that makes no a priori assumptions regarding the identity of the target concept has no rational basis for classifying any unseen instances.

Our previous learners were able to correctly classify new instances (if there are no errors in the training set and the target concept is included in $H$) because of the assumption that the target concept can be represented as conjunction of attributes. If this assumption is incorrect, at least some instances will be erroneously classified.

- The **inductive bias** of the Candidate-Elimination Algorithm using hypothesis space $H$ is that the target concept $c$ is included in $H$.
- The **inductive bias** of the Find-S Algorithm using hypothesis space $H$ is that the target concept $c$ is included in $H$ and that the solution is a maximally specific hypothesis (stronger inductive bias).

# Overfitting
Given an hypothesis, we call *training error*, and denote it with $error_{train}(h)$, the error on the training set, while we call *generalization error*, and denote it with $error_{D}(h)$, the error  made by hypothesis $h$ on the entire distribution $D$.

**Definition:** Given a hypothesis space $H$, a hypothesis $h \in H$ is said to overfit the training data if there exists some alternative hypothesis $h' \in H$ such that $h$ has smaller error than $h'$ over the training examples, but $h'$ has a smaller error than $h$ over the entire distribution of instances.

$$error_{train}(h) < error_{train}(h') \quad \wedge \quad error_{D}(h) > error_{D}(h')$$
$$accuracy_{train}(h) > accuracy_{train}(h') \quad \wedge \quad accuracy_{D}(h) < accuracy_{D}(h')$$

